# PyCity Schools Analysis

Your analysis here (from perspective of Chief Data Scientist for city's school district, to help the school board and mayor make strategic decisions regarding future school budgets and priorities):
  
* Across all 15 schools in the district, only 65% of students are passing both math and reading; 82% and 75% of students are passing reading only and math only, respectively.
* At the same time, comparison of the five top-performing and five bottom-performing schools, as well as performance by expeditures per student, reveal that lower-performing schools have larger budgets and spend more per student. This evidence can help rebutt claims that a lack of resources per se may be behind the underperformance of some schools in the district -- this is not the case. Indeed, the causation has likely run in the opposite direction, with lower-performing schools being allocated more funds on a per-pupil basis in an effort to offset other factors driving the education performance imbalance. Toward identifying root causes, and in turn potentially fruitful solutions, factors worth exploring as possibly contributing to the performance differential include school condition, teacher quality, student-to-teacher ratios, and dynamics in the home and broader community.
* In support of school size having some explantory power, one finds for both reading and math little performance difference between small and medium-sized schools but a substantial difference between medium and large schools. Additional data on teacher populations and class sizes could allow for more granular inference along this dimension. The evidence further points to a strong positive correlation between school type and student performance, with charter schools associated with a significant performance advantage on average and substantially higher performance on a percentage-passing basis. From the persepctive of delivering the most education value for the dollar, more attention should be given to how we might expand charter schools within the district or how elements of charter schools could be adapted to the benefit of district schools.
* With respect to performance as students progress from the 9th to 12th grades within a given school, the data consistently show little variation both over time and across the district's schools, suggesting that one's academic performance as they enter the 9th grade is a good predictor of their performance as they exit the 12th grade, that is, the effects of those factors driving differences in performance are persistent. This finding also highlights the importance of early education before pupils arrive to the 9th grade.  
* As a next step, I recommend analyzing performance data over time, toward: i) identifying shifting patterns if any, ii) sheddling light on the impact of prior efforts to boost students' performance district-wide, and iii) informing development of a 5-year plan the the of 100% passing rates on math and reading in all schools in the district within X years.

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
schoolDF = pd.read_csv(school_data_to_load)
studentDF = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
combinedDF = pd.merge(studentDF, schoolDF, how="left", on="school_name")

combinedDF.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [2]:
# Calculate the total number of unique schools in Combined DF
schoolCount = len(combinedDF["school_name"].unique())
schoolCount


15

In [3]:
# Calculate the total number of students in Combined DF
studentCount = combinedDF["Student ID"].count()
studentCount

39170

In [4]:
# Calculate the total district-wide budget -- sum school-level budgets in School DF
totalBudget = schoolDF["budget"].sum()
totalBudget


24649428

In [5]:
# Calculate the average math score -- take average across all students in Combined DF
aveMathScore = combinedDF["math_score"].mean()
aveMathScore


78.98537145774827

In [6]:
# Calculate the average reading score -- take average across all students in Combined DF
aveReadingScore = combinedDF["reading_score"].mean()
aveReadingScore


81.87784018381414

In [7]:
# Calculate the percentage of students who passed math (math scores >= 70) in Combined DF
passingMath = combinedDF[(combinedDF["math_score"] >= 70)]
numPassingMath = len(passingMath)

percentPassingMath = numPassingMath / float(studentCount) * 100
percentPassingMath


74.9808526933878

In [8]:
# Calculate the percentage of students who passed reading (reading scores >= 70) in Combined DF
passingReading = combinedDF[(combinedDF["reading_score"] >= 70)]
numPassingReading = len(passingReading)
percentPassingReading = numPassingReading / float(studentCount) * 100
percentPassingReading


85.80546336482001

In [9]:
# Calculate the percentage of students that passed math and reading in Combined DF
passingMathAndReading = combinedDF[
    (combinedDF["math_score"] >= 70) & (combinedDF["reading_score"] >= 70)
]
numPassingOverall = len(passingMathAndReading)
percentPassingOverall = numPassingOverall /  float(studentCount) * 100
percentPassingOverall


65.17232575950983

In [10]:
# Create a high-level snapshot of the district's key metrics in a DataFrame - make a dictionary of lists
districtSummary = pd.DataFrame(
    {"Total Schools": [schoolCount],
     "Total Students": [studentCount],
     "Total Budget": [totalBudget],
     "Average Math Score": [aveMathScore],
     "Average Reading Score": [aveReadingScore],
     "% Passing Math": [percentPassingMath],
     "% Passing Reading": [percentPassingReading],
     "% Overall Passing": [percentPassingOverall]
    }   
)

# Format the dataframe
districtSummary["Total Students"] = districtSummary["Total Students"].map("{:,}".format)
districtSummary["Total Budget"] = districtSummary["Total Budget"].map("${:,.2f}".format)
districtSummary["Average Math Score"] = districtSummary["Average Math Score"].map("{:,.2f}".format)
districtSummary["Average Reading Score"] = districtSummary["Average Reading Score"].map("{:,.2f}".format)
districtSummary["% Passing Math"] = districtSummary["% Passing Math"].map("{:,.2f}".format)
districtSummary["% Passing Reading"] = districtSummary["% Passing Reading"].map("{:,.2f}".format)
districtSummary["% Overall Passing"] = districtSummary["% Overall Passing"].map("{:,.2f}".format)

# Display the dataframe
districtSummary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary

In [11]:
# Look at School DF
schoolDF

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [12]:
# Extract each school's type from School DF
perSchoolType = schoolDF.set_index("school_name")["type"]
# perSchoolType

In [13]:
# Extract each school's student count from School DF
perSchoolNumStudents = schoolDF.set_index("school_name")["size"]
# Alternatively:
# perSchoolNumStudents = combinedDF["school_name"].value_counts()

# perSchoolNumStudents

In [14]:
# Extract total budget for each school from School DF
perSchoolBudget = schoolDF.set_index("school_name")["budget"]
# perSchoolBudget

In [15]:
# Calculate per capita spending for each school

schoolDF["per capita spending"] = schoolDF["budget"] / schoolDF["size"]  
perSchoolBudgetPerCap = schoolDF.set_index(["school_name"])["per capita spending"] 

# perSchoolBudgetPerCap

In [16]:
# Calculate average math test scores per school from Combined DF
perSchoolAveMathScoreDF = combinedDF.groupby("school_name")["math_score"].mean()

# Convert to series
perSchoolAveMathScore = perSchoolAveMathScoreDF
# perSchoolAveMathScore

In [17]:
# Calculate average reading test scores per school from Combined DF
perSchoolAveReadingScoreDF = combinedDF.groupby("school_name")["reading_score"].mean()

# Convert to series
perSchoolAveReadingScore = perSchoolAveReadingScoreDF
# perSchoolAveReadingScore

In [18]:
# Calculate the number of students per school with math scores of 70 or higher from Combined DF
perSchoolNumStudentsPassingMath = passingMath.groupby(["school_name"]).size()
#perSchoolNumStudentsPassingMath

In [19]:
# Calculate the number of students per school with reading scores of 70 or higher from Combined DF
perSchoolNumStudentsPassingReading = passingReading.groupby(["school_name"]).size()
# perSchoolNumStudentsPassingReading

In [20]:
# Calculate the number of students per school that passed both math and reading with scores of 70 or higher
perSchoolNumStudentsPassingMathAndReading = passingMathAndReading.groupby(["school_name"]).size()
# perSchoolNumStudentsPassingMathAndReading

In [21]:
# Calculate the passing rates
perSchoolPercentPassingMath = perSchoolNumStudentsPassingMath / perSchoolNumStudents * 100
perSchoolPercentPassingReading = perSchoolNumStudentsPassingReading / perSchoolNumStudents * 100
perSchoolPercentPassingOverall = perSchoolNumStudentsPassingMathAndReading / perSchoolNumStudents * 100
# print(perSchoolPercentPassingMath, perSchoolPercentPassingReading, perSchoolPercentPassingOverall)

In [22]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
school_names = combinedDF["school_name"].unique()
perSchoolSummary = pd.DataFrame(
    {"School Type": perSchoolType,
     "Total Students": perSchoolNumStudents,
     "Total School Budget": perSchoolBudget,
     "Per Student Budget": perSchoolBudgetPerCap,
     "Average Math Score": perSchoolAveMathScore,
     "Average Reading Score": perSchoolAveReadingScore,
     "% Passing Math": perSchoolPercentPassingMath,
     "% Passing Reading": perSchoolPercentPassingReading,
     "% Overall Passing": perSchoolPercentPassingOverall
    }   
)

# Formatting
perSchoolSummary.index.name = "School District Summary, by School"
perSchoolSummary["Total School Budget"] = perSchoolSummary["Total School Budget"].map("${:,.2f}".format)
perSchoolSummary["Per Student Budget"] = perSchoolSummary["Per Student Budget"].map("${:,.2f}".format)
perSchoolSummary["Average Math Score"] = perSchoolSummary["Average Math Score"].map("{:,.2f}".format)
perSchoolSummary["Average Reading Score"] = perSchoolSummary["Average Reading Score"].map("{:,.2f}".format)
perSchoolSummary["% Passing Math"] = perSchoolSummary["% Passing Math"].map("{:,.2f}".format)
perSchoolSummary["% Passing Reading"] = perSchoolSummary["% Passing Reading"].map("{:,.2f}".format)
perSchoolSummary["% Overall Passing"] = perSchoolSummary["% Overall Passing"].map("{:,.2f}".format)

# Display the DataFrame
perSchoolSummary


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
"School District Summary, by School",,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,89.23
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


## Highest-Performing Schools (by % Overall Passing)

In [23]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
topSchools = perSchoolSummary.sort_values(["% Overall Passing"], ascending=False)

topSchools.index.name = "Five Top-Performing Schools in District"
topSchools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Five Top-Performing Schools in District,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

In [24]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottomSchools = perSchoolSummary.sort_values(["% Overall Passing"], ascending=True)

bottomSchools.index.name = "Five Bottom-Performing Schools in District"
bottomSchools.head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Five Bottom-Performing Schools in District,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


## Math Scores by Grade

In [25]:
# Use the code provided to separate the data by grade
ninthGradersDF = combinedDF[combinedDF["grade"] == "9th"]
tenthGradersDF = combinedDF[combinedDF["grade"] == "10th"]
eleventhGradersDF = combinedDF[combinedDF["grade"] == "11th"]
twelfthGradersDF = combinedDF[combinedDF["grade"] == "12th"]

In [26]:
# Group by school_name and take the mean of the math_score column for each school
ninthGradeMathScoresDF = ninthGradersDF.groupby(["school_name"])["math_score"].mean()
tenthGradeMathScoresDF = tenthGradersDF.groupby(["school_name"])["math_score"].mean()
eleventhGradeMathScoresDF = eleventhGradersDF.groupby(["school_name"])["math_score"].mean()
twelfthGradeMathScoresDF = twelfthGradersDF.groupby(["school_name"])["math_score"].mean()

# Combine each of the scores above into a single DataFrame
mathScoresByGradeDF = pd.DataFrame({
    "9th": ninthGradeMathScoresDF,
    "10th": tenthGradeMathScoresDF,
    "11th": eleventhGradeMathScoresDF,
    "12th": twelfthGradeMathScoresDF
    }
)
                                    
# Minor data wrangling
mathScoresByGradeDF.index.name = "Math Scores by Grade, by School"
#mathScoresByGradeDF["9th"] = mathScoresByGradeDF["9th"].map("{:.2f}".format)
#mathScoresByGradeDF["10th"] = mathScoresByGradeDF["10th"].map("{:.2f}".format)
#mathScoresByGradeDF["11th"] = mathScoresByGradeDF["11th"].map("{:.2f}".format)
#mathScoresByGradeDF["12th"] = mathScoresByGradeDF["12th"].map("{:.2f}".format)

# Display the DataFrame
mathScoresByGradeDF


,9th,10th,11th,12th
"Math Scores by Grade, by School",,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

In [27]:
# Group by school_name and take the mean of the the reading_score column for each school
ninthGradeReadingScoresDF = ninthGradersDF.groupby(["school_name"])["reading_score"].mean()
tenthGradeReadingScoresDF = tenthGradersDF.groupby(["school_name"])["reading_score"].mean()
eleventhGradeReadingScoresDF = eleventhGradersDF.groupby(["school_name"])["reading_score"].mean()
twelfthGradeReadingScoresDF = twelfthGradersDF.groupby(["school_name"])["reading_score"].mean()

# Combine each of the scores above into a single DataFrame 
readingScoresByGradeDF = pd.DataFrame({
    "9th": ninthGradeReadingScoresDF,
    "10th": tenthGradeReadingScoresDF,
    "11th": eleventhGradeReadingScoresDF,
    "12th": twelfthGradeReadingScoresDF
    }
)

# Minor data wrangling
readingScoresByGradeDF.index.name = "Reading Scores by Grade, by School"
readingScoresByGradeDF["9th"] = readingScoresByGradeDF["9th"].map("{:.2f}".format)
readingScoresByGradeDF["10th"] = readingScoresByGradeDF["10th"].map("{:.2f}".format)
readingScoresByGradeDF["11th"] = readingScoresByGradeDF["11th"].map("{:.2f}".format)
readingScoresByGradeDF["12th"] = readingScoresByGradeDF["12th"].map("{:.2f}".format)

# Display the DataFrame
readingScoresByGradeDF


,9th,10th,11th,12th
"Reading Scores by Grade, by School",,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

In [28]:
# Establish the bins
spendingBins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [29]:
# Create a copy of the school summary for later aggregations
schoolSpendingDF = perSchoolSummary.copy()

In [30]:
# Use pd.cut on the perSchoolBudgetPerCapita series from above to categorize per student spending based on the bins
schoolSpendingDF["Expenditures Per Student"] = pd.cut(perSchoolBudgetPerCap, spendingBins, labels=labels)

# Convert numeric values that became strings in mapping above back to numbers
schoolSpendingDF["Average Math Score"] = schoolSpendingDF["Average Math Score"].astype(float)
schoolSpendingDF["Average Reading Score"] = schoolSpendingDF["Average Reading Score"].astype(float)
schoolSpendingDF["% Passing Math"] = schoolSpendingDF["% Passing Math"].astype(float)
schoolSpendingDF["% Passing Reading"] = schoolSpendingDF["% Passing Reading"].astype(float)
schoolSpendingDF["% Overall Passing"] = schoolSpendingDF["% Overall Passing"].astype(float)

schoolSpendingDF

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Expenditures Per Student
"School District Summary, by School",,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64,$585-630
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33,<$585
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20,$630-645
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29,$630-645
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60,$585-630
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53,$645-680
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,89.23,<$585
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51,$645-680
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54,$645-680


In [31]:
# Calculate averages for the desired columns; first, run groupby on spending ranges
spendingGroupedDF = schoolSpendingDF.groupby("Expenditures Per Student", observed=True)
spendingGroupedDF = spendingGroupedDF.mean(numeric_only=True)

# Next; calculate each series
spendingGroupedDF.index.name = "Student Performance by School Expenditures Per Student"
spendingTotalStudentsSeries = spendingGroupedDF["Total Students"] 
spendingMathSeries = spendingGroupedDF["Average Math Score"]
spendingReadingSeries = spendingGroupedDF["Average Reading Score"]
spendingPassingMathSeries = spendingGroupedDF["% Passing Math"]
spendingPassingReadingSeries = spendingGroupedDF["% Passing Reading"]
spendingPassingOverallSeries = spendingGroupedDF["% Overall Passing"]

spendingGroupedDF

,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Student Performance by School Expenditures Per Student,,,,,,
<$585,1592.000000,83.452500,83.932500,93.460000,96.610000,90.367500
$585-630,2291.750000,81.900000,83.155000,87.132500,92.717500,81.417500
$630-645,2830.500000,78.517500,81.625000,73.485000,84.392500,62.857500
$645-680,4104.333333,76.996667,81.026667,66.163333,81.133333,53.526667


In [32]:
# Assemble into DataFrame that shows student performance by spending per student at the school level
spendingSummaryDF = pd.DataFrame({
    "Average Math Score": spendingMathSeries,   
    "Average Reading Score": spendingReadingSeries,
    "% Passing Math": spendingPassingMathSeries,
    "% Passing Reading": spendingPassingReadingSeries,
    "% Overall Passing": spendingPassingOverallSeries
    }
)

# Minor data wrangling
spendingSummaryDF["Average Math Score"] = spendingSummaryDF["Average Math Score"].map("{:.2f}".format)
spendingSummaryDF["Average Reading Score"] = spendingSummaryDF["Average Reading Score"].map("{:.2f}".format)
spendingSummaryDF["% Passing Math"] = spendingSummaryDF["% Passing Math"].map("{:.2f}".format)
spendingSummaryDF["% Passing Reading"] = spendingSummaryDF["% Passing Reading"].map("{:.2f}".format)
spendingSummaryDF["% Overall Passing"] = spendingSummaryDF["% Overall Passing"].map("{:.2f}".format)

# Display results
spendingSummaryDF.index.name = "Student Performance by School Experditures per Student"
spendingSummaryDF


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Student Performance by School Experditures per Student,,,,,
<$585,83.45,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

In [33]:
# Establish the bins.
sizeBins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [34]:
# Create a copy of the school summary for later aggregations
schoolSizeDF = perSchoolSummary.copy()

In [35]:
# Use `pd.cut` on the perSchoolNumStudents series from earlier to categorize school size based on the bins.
schoolSizeDF["School Size"] = pd.cut(perSchoolNumStudents, sizeBins, labels=labels)

# Convert numeric values that became strings in mapping above back to numbers
schoolSizeDF["Average Math Score"] = schoolSizeDF["Average Math Score"].astype(float)
schoolSizeDF["Average Reading Score"] = schoolSizeDF["Average Reading Score"].astype(float)
schoolSizeDF["% Passing Math"] = schoolSizeDF["% Passing Math"].astype(float)
schoolSizeDF["% Passing Reading"] = schoolSizeDF["% Passing Reading"].astype(float)
schoolSizeDF["% Overall Passing"] = schoolSizeDF["% Overall Passing"].astype(float)

# Convert School Size to a string
schoolSizeDF["School Size"] = schoolSizeDF["School Size"].astype(str)

schoolSizeDF.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
"School District Summary, by School",,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60,Medium (1000-2000)


In [36]:
# Calculate averages for the desired columns; this scenario shows a more compact way to structure groupbys relative to prior scenario

# Calculate averages for the desired columns; first, run groupby on spending ranges
sizeGroupedDF = schoolSizeDF.groupby("School Size", observed=True)
sizeGroupedDF = sizeGroupedDF.mean(numeric_only=True)

# Next; calculate each series
sizeMathSeries = sizeGroupedDF["Average Math Score"]
sizeReadingSeries = sizeGroupedDF["Average Reading Score"]
sizePassingMathSeries = sizeGroupedDF["% Passing Math"]
sizePassingReadingSeries = sizeGroupedDF["% Passing Reading"]
sizeOverallPassingSeries = sizeGroupedDF["% Overall Passing"]

sizeGroupedDF

# Alternative formulation - skip display, continue to next step
#sizeMathSeries = schoolSizeDF.groupby(["School Size"])["Average Math Score"].mean()
#sizeReadingSeries = schoolSizeDF.groupby(["School Size"])["Average Reading Score"].mean()
#sizePassingMathSeries = schoolSizeDF.groupby(["School Size"])["% Passing Math"].mean()
#sizePassingReadingSeries = schoolSizeDF.groupby(["School Size"])["% Passing Reading"].mean()
#sizeOverallPassingSeries = schoolSizeDF.groupby(["School Size"])["% Overall Passing"].mean()

,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,,
Large (2000-5000),3657.375,77.745,81.34375,69.96375,82.76625,58.285
Medium (1000-2000),1704.400,83.374,83.86600,93.59800,96.79000,90.620
Small (<1000),694.500,83.820,83.92500,93.55000,96.10000,89.885


In [37]:
# Assemble into DataFrame that shows student performance by school size (small, medium, or large)
sizeSummaryDF = pd.DataFrame({
    "Average Math Score": sizeMathSeries,   
    "Average Reading Score": sizeReadingSeries,
    "% Passing Math": sizePassingMathSeries,
    "% Passing Reading": sizePassingReadingSeries,
    "% Overall Passing": sizeOverallPassingSeries
    }
)

# Minor data wrangling
sizeSummaryDF["Average Math Score"] = sizeSummaryDF["Average Math Score"].map("{:.2f}".format)
sizeSummaryDF["Average Reading Score"] = sizeSummaryDF["Average Reading Score"].map("{:.2f}".format)
sizeSummaryDF["% Passing Math"] = sizeSummaryDF["% Passing Math"].map("{:.2f}".format)
sizeSummaryDF["% Passing Reading"] = sizeSummaryDF["% Passing Reading"].map("{:.2f}".format)
sizeSummaryDF["% Overall Passing"] = sizeSummaryDF["% Overall Passing"].map("{:.2f}".format)

# Display results
sizeSummaryDF.index.name = "Student Performance by School Size"
sizeSummaryDF


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Student Performance by School Size,,,,,
Large (2000-5000),77.75,81.34,69.96,82.77,58.28
Medium (1000-2000),83.37,83.87,93.60,96.79,90.62
Small (<1000),83.82,83.93,93.55,96.10,89.89


## Scores by School Type

In [38]:
# Group the perSchoolSummary DataFrame by "School Type" and average the results.

# Create a copy of the school summary for later aggregations
schoolTypeDF = perSchoolSummary.copy()

# Convert numeric values that became strings in mapping above back to numbers
schoolTypeDF["Average Math Score"] = schoolTypeDF["Average Math Score"].astype(float)
schoolTypeDF["Average Reading Score"] = schoolTypeDF["Average Reading Score"].astype(float)
schoolTypeDF["% Passing Math"] = schoolTypeDF["% Passing Math"].astype(float)
schoolTypeDF["% Passing Reading"] = schoolTypeDF["% Passing Reading"].astype(float)
schoolTypeDF["% Overall Passing"] = schoolTypeDF["% Overall Passing"].astype(float)

# Convert School Size to a string
schoolTypeDF["School Type"] = schoolTypeDF["School Type"].astype(str)

# Calculate averages for the desired columns; first, run groupby on spending ranges
typeGroupedDF = schoolTypeDF.groupby("School Type", observed=True)
typeGroupedDF = typeGroupedDF.mean(numeric_only=True)

# Next; calculate each series
typeMathSeries = typeGroupedDF["Average Math Score"]
typeReadingSeries = typeGroupedDF["Average Reading Score"]
typePassingMathSeries = typeGroupedDF["% Passing Math"]
typePassingReadingSeries = typeGroupedDF["% Passing Reading"]
typeOverallPassingSeries = typeGroupedDF["% Overall Passing"]

typeGroupedDF

,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,,
Charter,1524.250000,83.472500,83.896250,93.620000,96.586250,90.431250
District,3853.714286,76.955714,80.965714,66.548571,80.798571,53.671429


In [39]:
# Assemble the new data by type into a DataFrame that shows student performance by school type (charter, district)
typeSummaryDF = pd.DataFrame({
    "Average Math Score": typeMathSeries,   
    "Average Reading Score": typeReadingSeries,
    "% Passing Math": typePassingMathSeries,
    "% Passing Reading": typePassingReadingSeries,
    "% Overall Passing": typeOverallPassingSeries
    }
)

# Minor data wrangling
typeSummaryDF["Average Math Score"] = typeSummaryDF["Average Math Score"].map("{:.2f}".format)
typeSummaryDF["Average Reading Score"] = typeSummaryDF["Average Reading Score"].map("{:.2f}".format)
typeSummaryDF["% Passing Math"] = typeSummaryDF["% Passing Math"].map("{:.2f}".format)
typeSummaryDF["% Passing Reading"] = typeSummaryDF["% Passing Reading"].map("{:.2f}".format)
typeSummaryDF["% Overall Passing"] = typeSummaryDF["% Overall Passing"].map("{:.2f}".format)

# Display results
typeSummaryDF.index.name = "Student Performance by School Type"
typeSummaryDF


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Student Performance by School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
